# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

## LLMChain

In [11]:
import ollama
ollama.list()

{'models': [{'name': 'gemma:2b-instruct',
   'model': 'gemma:2b-instruct',
   'modified_at': '2024-03-22T10:57:30.1963517+05:30',
   'size': 1678456656,
   'digest': 'b50d6c999e592ae4f79acae23b4feaefbdfceaa7cd366df2610e3072c052a160',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'gemma',
    'families': ['gemma'],
    'parameter_size': '3B',
    'quantization_level': 'Q4_0'}}]}

In [17]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [18]:
llm = ChatOllama(model="gemma:2b-instruct")

In [22]:
prompt = ChatPromptTemplate.from_template(
    "Create a catchy ad in one line for \
    a company that makes {product}?"
)

In [23]:
chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
product = "Queen Size Sheet Set"
print(chain.run(product))

Sure, here's a catchy ad for the Queen Size Sheet Set company:

**Sleep like royalty, without the hassle of finding the perfect fit.**


## SimpleSequentialChain

In [29]:
from langchain.chains import SimpleSequentialChain

In [30]:
llm = ChatOllama(model="gemma:2b-instruct")

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [31]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [34]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=False)

In [36]:
print(overall_simple_chain.run(product))

Sure, here's a 20 words description of the companies you provided:

1. The Empress Collection
2. The Royal Bedding Boutique
3. The Queen's Nook
4. The Pillow Palace
5. The Sleep Goddess
6. The Sleeping Queen
7. The Royal Silk Suite
8. The Sovereign Sheet Company
9. The Majestic Mattress


## SequentialChain

In [47]:
from langchain.chains import SequentialChain

In [55]:
llm = ChatOllama(temperature=0.9, model='gemma:2b-instruct')

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Summarize the content in the news article:"
    "\n\n{content}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="News"
                    )


In [56]:
second_prompt = ChatPromptTemplate.from_template(
    "why is the refinery refusing oil? from whom?"
    "\n\n{News}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="answer"
                    )


In [57]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{answer}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [58]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Using the news extract answer the questions and translate the news too:"
    "\n\n answers: {answer}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [60]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["content"],
    output_variables=["answer", "language","followup_message"],
    verbose=False
)

In [61]:
content = """All of India’s refiners are now refusing to take Russian crude carried on PJSC Sovcomflot tankers due to US sanctions, further complicating the trade that has flourished since the invasion of Ukraine two years ago.
Private and state-run processors including the biggest — Indian Oil Corp. — have stopped taking cargoes if they’re on Sovcomflot tankers, said people familiar with the matter. Refiners are scrutinizing the ownership of each ship to make sure they’re not affiliated with the company, or other sanctioned groups, they added, asking not to be named because the information is private."""
overall_chain.invoke(content)

c:\Users\kaila\DLAI\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'content': 'All of India’s refiners are now refusing to take Russian crude carried on PJSC Sovcomflot tankers due to US sanctions, further complicating the trade that has flourished since the invasion of Ukraine two years ago.\nPrivate and state-run processors including the biggest — Indian Oil Corp. — have stopped taking cargoes if they’re on Sovcomflot tankers, said people familiar with the matter. Refiners are scrutinizing the ownership of each ship to make sure they’re not affiliated with the company, or other sanctioned groups, they added, asking not to be named because the information is private.',
 'answer': "The news article explains that all of India's refiners have refused to take Russian crude carried on PJSC Sovcomflot tankers due to the US sanctions.",
 'language': 'The language of the review is English. It is a news article about a topic related to India and Russia, and the review is written in English.',
 'followup_message': "**Questions:**\n\n1. What is the main topic 

## Router Chain

In [8]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [9]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [11]:
from langchain.chains.router import MultiPromptChain
from langchain_community.chat_models import ChatOllama
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [12]:
import ollama
ollama.list()

{'models': [{'name': 'mistral-openorca:latest',
   'model': 'mistral-openorca:latest',
   'modified_at': '2024-03-24T22:54:42.2695876+05:30',
   'size': 4108928254,
   'digest': '12dc6acc14d0c4d3309e035cac6ff83e64c023aa774b21c41e342982699f436c',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': None,
    'parameter_size': '7B',
    'quantization_level': 'Q4_0'}},
  {'name': 'phi:latest',
   'model': 'phi:latest',
   'modified_at': '2024-03-24T22:34:59.1783023+05:30',
   'size': 1602463378,
   'digest': 'e2fd6321a5fe6bb3ac8a4e6f1cf04477fd2dea2924cf53237a995387e152ee9c',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'phi2',
    'families': ['phi2'],
    'parameter_size': '3B',
    'quantization_level': 'Q4_0'}}]}

In [13]:
llm = ChatOllama(temperature=0, model='mistral-openorca:latest')

In [14]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [15]:
destination_chains

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatOllama(model='mistral-openorca:latest', temperature=0.0)),
 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, \nanswer the component parts, and then put them togetherto answer the broader question.\n\nHere is a question:\n{input}'))]), llm=ChatOllama(model='mistral-openorca:latest', temp

In [16]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [17]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [18]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [19]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [20]:
chain.invoke("What is black body radiation?")

c:\Users\kaila\DLAI\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


' Black body radiation, also known as thermal radiation, is the electromagnetic radiation emitted by a black body, which is an idealized object that absorbs all the incoming radiation and does not reflect or transmit any. The temperature of a black body determines the wavelengths at which it emits the most energy. At higher temperatures, it emits more short-wavelength (high-frequency) radiation, such as visible light, ultraviolet, and X-rays. At lower temperatures, it emits longer-wavelength (lower-frequency) radiation, such as infrared and radio waves. The spectrum of black body radiation is described by the Planck distribution law, which relates the intensity of radiation to wavelength and temperature.<|im_end|>'

In [89]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': ''}
> Finished chain.


" Thank you for your kind words! I'll do my best to help you with your math question. Here it is:\n\nFind the derivative of the function f(x) = x^3 * sin(x).\n\nTo find the derivative of this function, we can use the sum rule and the product rule of differentiation. The sum rule states that if we have two functions g(x) and h(x), then the derivative of their sum is the sum of their derivatives: (g(x) + h(x))'.\n\nThe product rule states that if we have a function p(x) = u(x) * v(x), where u(x) and v(x) are differentiable functions, then the derivative of p(x) is given by:\n\np'(x) = u'(x) * v(x) + u(x) * v'(x)\n\nNow let's apply these rules to find the derivative of f(x) = x^3 * sin(x):\n\nFirst, we identify the two functions in our product: u(x) = x^3 and v(x) = sin(x).\n\nNext, we find the derivatives of each function:\n\nu'(x) = 3x^2\nv'(x) = cos(x)\n\nNow we can apply the product rule to find the derivative of our original function:\n\nf'(x) = u'(x) * v(x) + u(x) * v'(x)\n= (3x^2) 

In [90]:
chain.invoke("Who is the larget procuder of oil in the world")



> Entering new MultiPromptChain chain...
physics: {'input': 'Which country produces the most oil in the world?'}
> Finished chain.


" The United States produces the most oil in the world as of 2021. According to the U.S. Energy Information Administration, the U.S. surpassed Saudi Arabia and Russia in crude oil production in 2018. This trend has continued due to advancements in drilling technologies such as hydraulic fracturing (fracking) and horizontal drilling. However, it's important to note that rankings can change based on various factors, so always check the most recent data for accurate information."

Reminder: Download your notebook to you local computer to save your work.